# Task03. 下游任务的两阶段处理

# 1. 什么是两阶段 (two-stage) 处理

### 一般采集到的时序数据可能会有"瑕疵", 例如包含缺失值, 异常数据点和样本等等, 虽然这些"瑕疵"也属于数据的特性 (例如缺失值的模式可能代表了设备的异常情况), 但在一般的建模中是有害的不被我们需要的, 会阻碍算法的学习甚至让我们无法使用神经网络直接对其进行端到端的建模 (例如很多的时序预测算法和分类算法无法处理缺失值), 这时就需要我们在上游先对时序进行有特别目的的再处理, 然后进行下游任务(例如预测, 分类等)建模, 这样的建模处理分就被分为了两个阶段.

# 2. 使用TimesNet对Task02中插补的数据集进行分类